In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import random
import numpy as np
import json
from datetime import datetime
from pprint import pprint
from sklearn.model_selection import train_test_split
import os
from math import log
import collections

"""
    data_path : path for dataset
    output_column_name : Column Name with labels
    stochastic_param : list of 2 values float[p1,p2]-> p1 is the probability that right arm is user's choice,
                        p2 is the probability that the user might have chosen the wrong arm
"""
#For Cover_Type Dataset
def comp_dict(df):
            fin={}
            for index,row in df.iterrows():
                if row['user_id'] in fin.keys():
                    fin[row['user_id']].append(row['article_id'])

                else:
                    #if df.iloc[index]['user_id'] in self.users_test_and_training:
                    fin[row['user_id']]=[]
                    fin[row['user_id']].append(row['article_id'])

            for j in fin.keys():
                fin[j]=list(set(fin[j]))

            return fin
class Evaluator():

    def __init__(self, data_path, output_column_name,  print_params = {}, use_non_stationarity = False, sampling_fraction=0.1, stochastic_param = [1,0]):
        df_original = pd.read_csv('news_user.csv')
#         df_original = df_original.sort_values('timestamp')
        #df_original.iloc[:,~df_original.columns.duplicated()]
        df_original['article_id']=df_original['article_id'].astype(str)
        all_columns = list(df_original.columns)
        output_column_name = 'article_id'
        input_columns = all_columns[:]
        input_columns.remove(output_column_name)
        print('input cols : ', len(input_columns))
        x = df_original[input_columns].values
        self.print_params = print_params
        y=df_original[output_column_name].values
        #Scaling data
        scaler = MinMaxScaler()
        scaler.fit(x)
        x_scaled = scaler.transform(x)
        self.x_scaled=pd.DataFrame(x,columns = input_columns)
        self.x_scaled['article_id']=y
        col = ['user','app']
        self.final = pd.DataFrame(columns=col)

        col1 = ['user_id','article_id']
        self.final1 = df_original[col1]

        
        

        x_tr, x_test , y_tr, y_test = train_test_split(x_scaled,y,test_size=0.3,shuffle=False)
        x_train ,x_valid,y_train, y_valid = train_test_split(x_tr,y_tr,test_size=0.3,shuffle=False)
        self.x_tr = pd.DataFrame(x_tr, columns = input_columns)
        self.y_tr = pd.DataFrame(y_tr, columns = [output_column_name])
        

        self.df = pd.DataFrame(x_train, columns = input_columns)
        self.y_train = pd.DataFrame(y_train, columns = [output_column_name])
        


        self.df_valid = pd.DataFrame(x_valid, columns=input_columns)
        self.y_valid = pd.DataFrame(y_valid, columns = [output_column_name])


        #self.full_test = pd.DataFrame(x_test, columns=input_columns)
        #self.full_test['article_id'] = pd.DataFrame(y_test, columns = [output_column_name])
        #self.full_test = self.full_test.sort_values('user_id')
        self.df_test = pd.DataFrame(x_test, columns=input_columns)
        self.y_test = pd.DataFrame(y_test, columns = [output_column_name])




        self.total_train= self.df.copy(deep = True)
        self.total_train['article_id']=self.y_train.copy(deep = True)

        self.total_test= self.df_test.copy(deep = True)
        self.total_test['article_id']=self.y_test.copy(deep = True)

        


        self.dictionary_device = comp_dict(self.final)
        
#         self.dictionary_device = comp_dict(self.total_train)
#         self.dict_test_device= comp_dict(self.total_test)


        print ("Train Shape : ", self.df.shape ,self.y_train.shape)
        print ("Valid Shape :", self.df_valid.shape, self.y_valid.shape)
        print ("Test Shape :", self.df_test.shape,self.y_test.shape)

        print(self.y_test.iloc[0][0],self.y_test.iloc[-1][0])
        #self.df = pd.DataFrame(x_scaled, columns=input_columns)
        #self.df[output_column_name] = df_original[output_column_name]
        
        #Taking a subset of dataset
        #self.df = self.df.iloc[0:150000]

        self.arm_column_name = output_column_name
        self.logging_frequency = 1000
        self.number_features_in_context = len(input_columns)
        self.list_arms = df_original[self.arm_column_name].unique().tolist()
        #self.list_arms=self.list_arms[1:]
        self.num_arms = len(self.list_arms)
        print(self.num_arms)
        #To simulate stochasticity set this paramater
        self.stochastic_param = stochastic_param

        random.seed(9001)

        #Initially, the correct arm is the right arm, later this will change to simulate non stationarity
        self.dict_true_arm_to_right_arm = {arm:arm for arm in self.list_arms}
        #print(type(list(self.dict_true_arm_to_right_arm.keys())[0]))
        self.use_non_stationarity = use_non_stationarity
        self.non_stationarity_steps = 50000
        self.non_stationarity_param = 0
        self.device_app ={}

        print(len(set(self.df_test['user_id'].unique())))
        print(len(set(self.x_tr['user_id'].unique())))
        #print(list(set(self.df_test['user_id'].unique()).intersection(set(self.x_tr['user_id'].unique())))

        self.users_test_and_training = list(set(self.df_test['user_id'].unique()).intersection(set(self.x_tr['user_id'].unique())))

        print("Length of user_test_and_training:%d" % len(self.users_test_and_training))

        print ("Length of tesing" , len(self.df_test['user_id'].unique()))

                #Take only random user_sample of users for evaluations

        self.users_test_sample = self.users_test_and_training #self.remove_percentage(users_test_and_training, percentage)

    
        

    def get_list_arms(self):
        return self.list_arms

    def get_number_features_in_context(self):
        return self.number_features_in_context

    def get_dataset_shape(self):
        return self.df.shape

    def final_dict(self,final):
        #final=self.device_app
        
        
        dict_f={}
        for device,app_list in final.items():
            #print(device,app_list)
            dict_f[device]=[]
            
            p=len(app_list)-1
            while(p>=0 and len(dict_f[device])<3):
                if(app_list[p] not in dict_f.values()):
                    dict_f[device].append(app_list[p])
                    p-=1
        
            #print(final_rec)
            counter=collections.Counter(app_list)
            print(counter)          

            c=counter.most_common(3)
            m=-1
            for k,l in c:
                if (l>1 and k not in dict_f[device]):
                    dict_f[device][m]= k
                    m=m-1
                    
        #print(dict_f)
        final_rec= pd.Series(dict_f, name='App')
        final_rec.index.name = 'User'
        #print(final_rec)
        return final_rec,dict_f

    def evaluate(self, policy):

        start_time = datetime.now()
        rewards = 0
        rewards_test=0
        total_possible_reward = len(self.df_test) + len(self.df_valid)
        evaluated = 0
        evaluated_test=0
        match_found = False

#################################################TRAINING##################################################################
        #BATCH FIT TRAINING ROWS
        for index, row in self.df.iterrows():
            state = row.tolist()[:]
            right_arm = self.y_train.iloc[index][0]
            policy.update_model(1,right_arm,state, True)
           
            for i in range(3):
                a=random.choice(self.list_arms)
                if(a!=right_arm):
                    policy.update_model(0,a,state, True)

            #for a in self.list_arms:
                #if(a!=right_arm):
                    #policy.update_model(0,right_arm,state, True)
                    
################################################VALIDATION####################################################################
                    
        #PARTIAL FIT TRAINING ROAD
        for index, row in self.df_valid.iterrows():
            state = row.tolist()[:]
            y_value = self.y_valid.iloc[index][0]
            #print (state)
            selected_action = policy.get_action(state)
            reward = 0
            # Cycle classes to simulate non-stationarity
            if row['user_id'] not in self.dictionary_device.keys():
                self.dictionary_device[row['user_id']] = []

            if y_value not in self.dictionary_device[row['user_id']]:
                self.dictionary_device[row['user_id']].append(y_value)
            right_arm=self.dictionary_device[row['user_id']]
            #self.y_valid.iloc[index][0]
            #print('right arm',right_arm)
            #print(type(right_arm))
            #right_arm = (self.dict_true_arm_to_right_arm[row[self.arm_column_name]] + self.non_stationarity_param)% self.num_arms + 1
            #print('dictionary',self.dict_true_arm_to_right_arm)
            #print(right_arm)

            # Use probability p for stochasticity
            p1 = round(random.uniform(0,1), 1)
            p2 = round(random.uniform(0,1), 1)
            
            match_found = False
            for i in range(5):
                reward=0
                
                if selected_action[i] == y_value : #and p1>(1-self.stochastic_param[0]):
                    match_found = True
                    reward = 1
                    #print('rightarm')

                #elif selected_action[i] not in right_arm and p2<self.stochastic_param[1]:
                    #reward = 1
                #print ("Out of loop")
                
                #print ("rewards", rewards)
                policy.update_model(reward,selected_action[i])

            if match_found:
                rewards += 1

            evaluated += 1

            if self.use_non_stationarity and evaluated % self.non_stationarity_steps == 0:
                self.non_stationarity_param += 1
                print (self.non_stationarity_param)


            if evaluated % self.logging_frequency == 0 and evaluated!=0:
                print("---------------------------------------------------------------------------------------------------------- ")
                print('evaluated and frequency reached %d' % evaluated)
                print('accuracy %f' % ( float(rewards) / evaluated ))
                print("----------------------------------------------------------------------------------------------------------- ")

##################################################TESTING##################################################################
        precision=0
        ndgc=0
        dict_recall= {}
        dict_impressions = {}
        dict_clicks = {}
        for index, row in self.df_test.iterrows():
            
            if (row['user_id'] in self.users_test_sample):

                state = row.tolist()[:]
                y_value = self.y_test.iloc[index][0]
                #print (state)
                selected_action = policy.get_action(state)
                if(row['user_id'] not in self.device_app.keys()):
                    self.device_app[row['user_id']]=[]
                for j in range(len(selected_action)):
                    if selected_action[j] not in self.device_app[row['user_id']]:
                        self.device_app[row['user_id']].append(selected_action[j])
                
                reward = 0

                print("selected_action", selected_action[0],selected_action[1],selected_action[2])
                # Cycle classes to simulate non-stationarity

                if row['user_id'] not in self.dictionary_device.keys():
                    self.dictionary_device[row['user_id']] = []

                if y_value not in self.dictionary_device[row['user_id']]:
                    self.dictionary_device[row['user_id']].append(y_value)

                right_arm=self.dictionary_device[row['user_id']]
            
            # Use probability p for stochasticity
                p1 = round(random.uniform(0,1), 1)
                p2 = round(random.uniform(0,1), 1)
                ps=0
                ps_sum = 0
                num_ndgc=0
                d_ndgc=0
                k_ndgc=0
                if (row['user_id'] not in dict_recall.keys()):
                    dict_recall[row['user_id']]=[]

                match_found = False
                
                for i in range(5):
                    reward=0
                    if selected_action[i] not in dict_impressions.keys():
                        dict_impressions[selected_action[i]] = 0
                        dict_clicks[selected_action[i]] = 0
                    dict_impressions[selected_action[i]] += 1
                    
                    if selected_action[i] in right_arm: #and p1>(1-self.stochastic_param[0]):
                        
                        ps+=1
                        ps_sum += ps/(i+1)
                        if(i+1==1):
                            d_ndgc+=1
                        else:
                            d_ndgc+=1/(log(i+1))
                        num_ndgc = num_ndgc+1
                        

                        if(selected_action[i] not in dict_recall[row['user_id']]):
                            dict_recall[row['user_id']].append(selected_action[i])
                    
                    if selected_action[i] in right_arm:
                        
                        match_found = True
                        reward = 1
                    
                    policy.update_model(reward,selected_action[i])
                    
                if y_value not in dict_clicks.keys():
                    dict_clicks[y_value] = 0
                dict_clicks[y_value] +=1
                
                
                for i in range(num_ndgc):
                    if(i+1 == 1):
                        k_ndgc+=1
                    else:
                        k_ndgc+= 1/(log(i+1))

                if match_found:
                    rewards += 1
                    rewards_test += 1
                evaluated += 1
                evaluated_test+=1
                precision =precision+ ps_sum/5
                if(k_ndgc!=0):
                    ndgc = ndgc + (d_ndgc/k_ndgc)

                if self.use_non_stationarity and evaluated % self.non_stationarity_steps == 0:
                    self.non_stationarity_param += 1
                    print (self.non_stationarity_param)


                if evaluated % self.logging_frequency == 0 and evaluated!=0:
                    print("---------------------------------------------------------------------------------------------------------- ")
                    print('evaluated and frequency reached %d' % evaluated)
                    print('accuracy %f' % ( float(rewards) / evaluated ))
                    print("----------------------------------------------------------------------------------------------------------- ")
            #print("evaluated",evaluated)
            #print("rewards",rewards)

            # every n steps, cycle classes
        
                #print('right_arm %d' % right_arm)
                #print(self.print_params)
        
        self.final['user']=self.device_app.keys()
        self.final['app']=self.device_app.values()

        #Recall
        recall=0
        for key, value in dict_recall.items():
            recall = recall + (len(value)/len(self.dictionary_device[key]))
        ctr = 0 
        for key, value in dict_impressions.items():
            ctr += dict_clicks[key]/value
        print ('recall')
        print (recall%len(dict_recall.keys()))

        print ("precision")
        print (precision/self.df_test.shape[0])
        
        print ("NDGC")
        print (ndgc/self.df_test.shape[0])
        
        print("CTR")
        print(ctr/len(dict_impressions.keys()))


#         print(len(self.device_app))
        print(rewards)
        print(total_possible_reward)
        print("accuracy", float(rewards)/total_possible_reward)

        print("---")
        print(rewards_test)
        print(evaluated_test)
        print("accuracy", float(rewards_test)/evaluated_test)

        end_time = datetime.now()
        print ("Total time taken by ",str(policy)," = ",end_time-start_time)
#         print (self.print_params)
        return self.device_app




            

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import matplotlib as mpl
import seaborn as sns
import random
#from eval_3_modified import Evaluator
from sklearn.tree import DecisionTreeRegressor
import itertools
import operator
from sklearn import linear_model
from sklearn.neural_network import MLPRegressor
#from imblearn.over_sampling import SMOTE

import tensorflow as tf
import sys


class TFNeuralNetworkRegressor():

	def multilayer_perceptron(self, x, weights, biases, keep_prob):
		# Hidden fully connected layer1
		layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
		layer_1 = tf.nn.tanh(layer_1)
		# Hidden fully connected layer2
		layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
		layer_2 = tf.nn.relu(layer_2)
		# Hidden fully connected layer2
# 		layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
# 		layer_3 = tf.nn.relu(layer_3)
		# Output fully connected layer with a neuron for each class
		out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
		return out_layer

	def __init__(self, n_input, arm_name, model_number, training_dropout = 1.0, prediction_dropout = 1.0):

		self.n_input = n_input
		self.experience = []
		self.model_number = arm_name
		self.training_dropout = training_dropout
		self.prediction_dropout = prediction_dropout

		# Network Parameters
		n_hidden_1 = 60 # 1st layer number of neurons
		n_hidden_2 = 40 # 2nd layer number of neurons
# 		n_hidden_3 = 40 # 3rd layer number of neurons

		weights = {
			'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
			'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
# 			'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3])),
			'out': tf.Variable(tf.random_normal([n_hidden_2, 1]))
		}
		biases = {
			'b1': tf.Variable(tf.random_normal([n_hidden_1])),
			'b2': tf.Variable(tf.random_normal([n_hidden_2])),
# 			'b3': tf.Variable(tf.random_normal([n_hidden_3])),
			'out': tf.Variable(tf.random_normal([1]))
		}
			
			
		 
			

		self.keep_prob = tf.placeholder("float", name="keep_prob")

		self.x = tf.placeholder("float", [None, n_input])
		self.y = tf.placeholder("float", [None, 1])

		self.predictions = self.multilayer_perceptron(self.x, weights, biases,
													  self.keep_prob)

		self.cost = tf.reduce_mean(tf.square(self.predictions - self.y))

		self.train = tf.train.AdamOptimizer(0.008).minimize(self.cost)

		self.sess = tf.Session()

		self.sess.run(tf.global_variables_initializer())


	def get_samples_from_experience(self, must_have_row, must_have_column, n_samples=50):
		num_to_sample = min(len(self.experience), n_samples)
		sample = random.sample(self.experience, num_to_sample)
		x = [must_have_row]
		y = [must_have_column]
		for row in sample:
			x.append(row[0])
			y.append(row[1])
		x = np.array(x).reshape(num_to_sample + 1, self.n_input)
		y = np.array(y).reshape(num_to_sample + 1, 1)

		if num_to_sample > 500:
			sm = SMOTE(random_state=42)
			X_res, y_res = sm.fit_sample(x, y)
			y_res = y_res.reshape(y_res.shape[0], 1)

		else:
			X_res, y_res = x, y

		return X_res, y_res

	def fit(self, X, Y):
		self.experience.append((X[0], Y))
		# if len(self.experience)>1000:
		#     self.experience.pop(0)
		# forcing the experience sample to return this piece of information

		# Y = [Y]
		for i in range(50):
			x, y = self.get_samples_from_experience(X[0], Y)
			train = self.sess.run([self.train],
								feed_dict = {self.x:x, self.y:y, self.keep_prob:self.training_dropout})

	def partial_fit(self, X, Y):
		# print "Fitting", self.model_number
		self.fit(X, Y)

	def batch_fit(self, X, Y, iter = 1):
		# print "Fitting : ", self.model_number
		for i in range(0,iter):
			train = self.sess.run([self.train],
								feed_dict = {self.x:X, self.y:Y, self.keep_prob:self.training_dropout})

	def predict(self, X, verbose=False):
		preds = self.sess.run([self.predictions],
					 feed_dict = {self.x:X, self.keep_prob : self.prediction_dropout})
		if verbose:
			print(preds)
		return preds
    
class PerArmModelPolicy():
    def __init__(self, list_arms, number_features_in_context, training_dropout, prediction_dropout, epsilon, number_models_per_arm=1,
				switch_off_epsilon_exploration=True, use_epsilon_decay=True):
        self.list_arms = list_arms
        self.last_action = None
        self.last_reward = None
        self.last_state = None
        self.last_model = None

        self.experience = {}
        self.dict_arm_to_model = {}
        # self.epsilon_percentage = 10
                #If Using Batch Training
        self.batch_size = 0
        self.mini_batch = 500
        self.train_batch_size = 50000

		# if self.batch_size:
        self.iter = 0
        self.dict_update_hist = {}
        for arm in list_arms:
            self.dict_arm_to_model[arm] = []
            for i in range(number_models_per_arm):
                m = TFNeuralNetworkRegressor(number_features_in_context, arm, str(i), training_dropout, prediction_dropout)
                self.dict_arm_to_model[arm].append(m)

		# for epsilon decay
        self.num_feedback_received = 0
        self.decay = 50
        self.switch_off_epsilon_exploration = switch_off_epsilon_exploration
        self.use_epsilon_decay = use_epsilon_decay
        self.default_epsilon = epsilon
    def get_best_arm(self, state):
        best_score1 = 0
        best_score2 = 0
        best_score3 = 0
        best_score4 = 0
        best_score5 = 0

        best_arm1 = None
        best_arm2 = None
        best_arm3 = None
        best_arm4 = None
        best_arm5 = None

        best_model1 = None
        best_model2 = None
        best_model3 = None
        best_model4 = None
        best_model5 = None

        s=[best_score1,best_score2,best_score3, best_score4, best_score5]
        a=[best_arm1, best_arm2, best_arm3 , best_arm4, best_arm5]
        m=[best_model1 ,best_model2 , best_model3 ,best_model4, best_model5]

        #score_to_model = {}

        for arm, models in self.dict_arm_to_model.items():
			# score = model.predict([state])[0]
            try:
				# randomly sample a model from the list of models
                model = random.choice(models)
                score = model.predict([state])[0]
                #score_to_arm[score] = arm
				# print(score)`
                if score > best_score1 or best_arm1 is None:
                    best_score5 = best_score4
                    best_score4 = best_score3
                    best_score3 = best_score2
                    best_score2 = best_score1
                    best_score1 = score

                    best_arm5= best_arm4
                    best_arm4 = best_arm3
                    best_arm3= best_arm2
                    best_arm2 = best_arm1
                    best_arm1 = arm

                    best_model5 = best_model4
                    best_model4 = best_model3
                    best_model3 = best_model2
                    best_model2 = best_model1
                    best_model1 = model



                elif score > best_score2 and score < best_score1 :
                    best_score5 = best_score4
                    best_score4 = best_score3
                    best_score3 = best_score2
                    best_score2 = score


                    best_arm5= best_arm4
                    best_arm4 = best_arm3
                    best_arm3= best_arm2
                    best_arm2 = arm


                    best_model5 = best_model4
                    best_model4 = best_model3
                    best_model3 = best_model2
                    best_model2 = model

                elif score > best_score4 and score < best_score3:
                    best_score5 = best_score4
                    best_score4 = score

                    best_arm5= best_arm4
                    best_arm4 = arm


                    best_model5 = best_model4
                    best_model4 = model

                elif score > best_score4 and score < best_score5:
                    best_score5 = best_score4
                    best_score4 = score


                    best_arm5= best_arm4
                    best_arm4 = arm

                    best_model5 = best_model4
                    best_model4 = model

                else: #score > best_score5:
                    best_score5 = score
                    best_arm5= arm
                    best_model5 = model


            except Exception as e:
                print('model %s not ready yet' % arm)
                print('----------------------------------')
				# print(repr(e))
				# sys.exit(1)
                return s,a,m
		#print (type(best_arm))
        s=[best_score1,best_score2,best_score3, best_score4, best_score5]
        a=[best_arm1, best_arm2, best_arm3 , best_arm4, best_arm5]
        m=[best_model1 ,best_model2 , best_model3 ,best_model4, best_model5]
        return s,a,m

    def get_action(self, state):
        explore_action=[]
        for i in range(5):
            explore_action.append(random.choice(list_arms))
			
        best_score, best_action, best_model = self.get_best_arm(state)
        self.num_feedback_received += 1
        if self.use_epsilon_decay:
            epsilon = self.get_epsilon()
        else:
            epsilon = self.default_epsilon
		# if (not self.switch_off_epsilon_exploration) and (np.random.random() <= epsilon or best_action is None):
        if np.random.random() <= epsilon or best_action[0] is None:
            action = explore_action
        else:
			# print('chose best action')
            action = best_action
		# print(action)
		#self.last_action = action
		#print('action',self.last_action)
        self.last_state = state
        self.last_model = best_model
        return action


    def update_model(self, reward, last_action = None, last_state = [], update_in_train = False):
		# Immediate Updates for incremental training
        if not last_action:
            last_action = self.last_action
        if not last_state:
            last_state = self.last_state
        self.last_reward = reward

        if self.batch_size==0 and update_in_train == False and last_action!=None:
            self.last_reward = reward
            self.dict_arm_to_model[last_action][0].partial_fit([last_state], [self.last_reward])

		#Batch Updates for Li Hong Matching
        if update_in_train :
            if(self.iter == 0 or self.iter%self.train_batch_size != 0):
                #for i in range(0,self.num_locations):
                if last_action not in self.dict_update_hist.keys():
                        self.dict_update_hist[last_action] = []
                        self.dict_update_hist[last_action].append([])
                        self.dict_update_hist[last_action].append([])
                        #Storing Location
                        #self.dict_update_hist[last_actions].append(i)
                        self.dict_update_hist[last_action][0].append(last_state)
                        self.dict_update_hist[last_action][1].append([self.last_reward])
                else:
                    self.dict_update_hist[last_action][0].append(last_state)
                    self.dict_update_hist[last_action][1].append([self.last_reward])
            else:
                print( "Update Number: ", self.iter)
                for iteration in range(1,7001):
                    if(iteration%1000 == 0):
                        print ("Iteration Number : ", iteration)
                    for offer, update in self.dict_update_hist.items():
                        num_to_sample = min(len(update[0]), self.mini_batch)
                        random_indices = random.sample(range(0, len(update[0])), num_to_sample)
                        x_train = []
                        y_train = []
                        for index in random_indices:
                            x_train.append(update[0][index])
                            y_train.append(update[1][index])
                        self.dict_arm_to_model[offer][0].batch_fit(x_train, y_train)

                        #self.dict_per_location_offer_model[update[2]][offer].batch_fit(x_train, y_train)
            self.iter+=1


    def get_epsilon(self):
        """Produce epsilon"""
        total = self.num_feedback_received
        return float(self.decay) / (total + float(self.decay))



training_dropout, prediction_dropout, epsilon= 0.5, 0.5, 0.0000001
use_non_stationarity = False

print_params = {}
print_params["training_dropout"] = training_dropout
print_params["prediction_dropout"] = prediction_dropout
print_params["epsilon"] = epsilon

#Change these in the function get_samples_from_experience too
print_params["Sampling"] = 50
print_params["Number of times fit"] = 10

print_params["use_non_stationarity"] = use_non_stationarity

switch_off_epsilon_exploration = True
forest_cover_evaluator = Evaluator('news_user.csv', 'article_id', print_params, use_non_stationarity)
list_arms = forest_cover_evaluator.get_list_arms()
number_features_in_context = forest_cover_evaluator.get_number_features_in_context()
per_arm_model_policy = PerArmModelPolicy(list_arms, number_features_in_context, training_dropout, prediction_dropout, epsilon,1,
										switch_off_epsilon_exploration,
										False)
forest_cover_evaluator.evaluate(per_arm_model_policy)

input cols :  59
Train Shape :  (89421, 59) (89421, 1)
Valid Shape : (38324, 59) (38324, 1)
Test Shape : (54749, 59) (54749, 1)
206934 140528
40
44321
84459
Length of user_test_and_training:21809
Length of tesing 44321
Instructions for updating:
Colocations handled automatically by placer.
Update Number:  50000
Iteration Number :  1000
Iteration Number :  2000
Iteration Number :  3000
Iteration Number :  4000
Iteration Number :  5000
Iteration Number :  6000
Iteration Number :  7000
Update Number:  100000
Iteration Number :  1000
Iteration Number :  2000
Iteration Number :  3000
Iteration Number :  4000
Iteration Number :  5000
Iteration Number :  6000
Iteration Number :  7000
Update Number:  150000
Iteration Number :  1000
Iteration Number :  2000
Iteration Number :  3000
Iteration Number :  4000
Iteration Number :  5000
Iteration Number :  6000
Iteration Number :  7000
Update Number:  200000
Iteration Number :  1000
Iteration Number :  2000
Iteration Number :  3000
Iteration Number :

---------------------------------------------------------------------------------------------------------- 
evaluated and frequency reached 25000
accuracy 0.874920
----------------------------------------------------------------------------------------------------------- 
---------------------------------------------------------------------------------------------------------- 
evaluated and frequency reached 26000
accuracy 0.876308
----------------------------------------------------------------------------------------------------------- 
---------------------------------------------------------------------------------------------------------- 
evaluated and frequency reached 27000
accuracy 0.877815
----------------------------------------------------------------------------------------------------------- 
---------------------------------------------------------------------------------------------------------- 
evaluated and frequency reached 28000
accuracy 0.878107
-----------------

selected_action 119592 123290 156625
selected_action 29953 123290 160474
selected_action 225055 236444 156625
selected_action 235132 236444 156625
selected_action 313920 236951 156625
selected_action 284096 236444 156625
selected_action 225055 156625 202557
selected_action 140528 156625 202557
selected_action 123290 156625 236444
selected_action 29953 156625 236207
selected_action 285343 236444 156625
selected_action 236951 236444 156625
selected_action 119592 236444 156625
selected_action 215861 156625 236444
selected_action 313920 285343 236444
selected_action 338339 160474 236444
selected_action 182394 156625 236444
selected_action 284096 236444 285343
selected_action 123290 123826 283505
selected_action 337143 285343 123826
selected_action 215861 123818 283505
selected_action 87224 156625 337143
selected_action 95972 156625 236444
selected_action 202557 236444 156625
selected_action 236951 156625 285343
selected_action 337143 123290 236444
selected_action 123290 236444 236951
selec

selected_action 119592 236444 156625
selected_action 140528 156625 96663
selected_action 288435 285343 283505
selected_action 161425 156625 158082
selected_action 29953 123818 285343
selected_action 285343 123818 156625
selected_action 225019 39894 156625
selected_action 288435 123290 285343
selected_action 236951 156625 160474
selected_action 158082 160474 156625
selected_action 284096 236444 156625
selected_action 95972 156625 283505
selected_action 288435 123290 156625
selected_action 87224 123818 156625
selected_action 215861 156625 236444
selected_action 87224 236444 156625
selected_action 182394 123818 236444
selected_action 235132 156625 160474
selected_action 236444 156625 123818
selected_action 215861 123290 283505
selected_action 338339 156625 236444
selected_action 70591 156625 225019
selected_action 87224 123290 236444
selected_action 214800 236444 160474
selected_action 202557 123818 236951
selected_action 206934 156625 236444
selected_action 140528 156625 225055
selected_

selected_action 225019 123818 156625
selected_action 284096 156625 87224
selected_action 283505 156625 285343
selected_action 236444 276946 156625
selected_action 214800 156625 123826
selected_action 123826 123818 156625
selected_action 123290 123818 236444
selected_action 235132 236951 156625
selected_action 225019 123818 236951
selected_action 236951 236444 156625
selected_action 96663 95972 236444
selected_action 285343 236444 156625
selected_action 215861 123290 236444
selected_action 140528 236444 160474
selected_action 285343 236444 156625
selected_action 337143 236951 156625
selected_action 124176 123290 236444
selected_action 313920 123826 70591
selected_action 123290 156625 236444
selected_action 140528 236951 156625
selected_action 95972 236951 285343
selected_action 337143 236444 156625
selected_action 285343 156625 337143
selected_action 29953 236951 156625
selected_action 284096 156625 285343
selected_action 236951 236444 156625
selected_action 236951 236444 285343
selecte

...

selected_action 285343 95972 123290
selected_action 123826 160940 124176
selected_action 29953 236951 124176
selected_action 140528 283505 124176
selected_action 337143 96663 124176
selected_action 283505 160474 96663
selected_action 283392 285343 283505
selected_action 236207 285343 236951
selected_action 235132 285343 160474
selected_action 202557 285343 337143
selected_action 95972 123818 236951
selected_action 288435 236951 70591
selected_action 87224 215861 140528
selected_action 123818 123290 285343
selected_action 236207 123290 123818
selected_action 288435 160474 236951
selected_action 225019 160474 124176
selected_action 87224 285343 124176
selected_action 313920 96663 182394
selected_action 123818 123290 123826
selected_action 225055 160940 96663
selected_action 140528 236444 124176
selected_action 160474 160940 236951
selected_action 313920 236951 160940
selected_action 288435 123818 285343
selected_action 119592 285343 225019
selected_action 160474 140528 225019
selected_ac

selected_action 206934 123818 123826
selected_action 285343 215861 284096
selected_action 123818 123290 160940
selected_action 313920 236207 236444
selected_action 215861 95972 283505
selected_action 337143 160474 123826
selected_action 235132 285343 160474
selected_action 119592 96663 225019
selected_action 95972 285343 123290
selected_action 119592 160474 285343
selected_action 215861 95972 140528
selected_action 119592 96663 225019
selected_action 235132 283505 285343
selected_action 123818 123290 123826
selected_action 158082 123818 123826
selected_action 124176 215861 123290
selected_action 29953 160940 236951
selected_action 39894 95972 236951
selected_action 160474 160940 96663
selected_action 123290 123818 236951
selected_action 214800 215861 158082
selected_action 202557 123290 123818
selected_action 123290 236444 160940
selected_action 313920 160940 236444
selected_action 338339 123290 236444
selected_action 96663 95972 236444
selected_action 140528 123818 96663
selected_acti

selected_action 288435 123818 160474
selected_action 276946 283505 236444
selected_action 29953 236444 96663
selected_action 119592 285343 123826
selected_action 124176 123290 236444
selected_action 313920 283505 236444
selected_action 284096 96663 158082
selected_action 160474 236444 284096
selected_action 283505 236444 160940
selected_action 236444 283505 225019
selected_action 124176 123818 160940
selected_action 236444 283505 160474
selected_action 215861 182394 160474
selected_action 123290 123818 140528
selected_action 161425 215861 236444
selected_action 87224 285343 123826
selected_action 160940 215861 123818
selected_action 119592 96663 285343
selected_action 214800 160940 285343
selected_action 236444 215861 160474
selected_action 160474 160940 96663
selected_action 236207 160940 158082
selected_action 313920 123818 236444
selected_action 225055 160940 96663
selected_action 236951 236444 96663
selected_action 214800 236951 96663
selected_action 123818 123290 236444
selected_a

selected_action 160940 236951 96663
selected_action 39894 160940 236444
selected_action 29953 236444 160940
selected_action 160474 215861 236444
selected_action 313920 140528 160940
selected_action 39894 95972 215861
selected_action 236951 95972 236444
selected_action 39894 95972 236444
selected_action 236444 313920 225019
selected_action 29953 236951 225019
selected_action 123290 123818 160474
selected_action 95972 160474 236444
selected_action 160940 236444 96663
selected_action 96663 95972 236444
selected_action 236444 158082 236951
selected_action 225019 236444 160474
selected_action 119592 39894 236444
selected_action 161425 236444 236207
selected_action 161425 236207 283505
selected_action 123818 140528 96663
selected_action 215861 123818 236444
selected_action 160940 236207 283505
selected_action 140528 236951 225019
selected_action 70591 123290 236444
selected_action 276946 236444 236951
selected_action 119592 236444 123818
selected_action 123826 276946 236444
selected_action 2

{0.2773497020675644: ['160940',
  '156625',
  '124176',
  '206934',
  '39894',
  '338339',
  '236444',
  '140528',
  '225019'],
 0.03704953978420029: ['123826',
  '236444',
  '283505',
  '156625',
  '39894',
  '123290',
  '160474',
  '123818',
  '276946'],
 0.0483629201095103: ['236444',
  '156625',
  '123826',
  '283392',
  '39894',
  '123290',
  '123818',
  '96663',
  '236951'],
 0.09067118417303618: ['225055', '156625', '285343', '236444', '39894'],
 0.2884772617469618: ['87224', '123818', '283505', '276946', '39894'],
 0.5523178028566827: ['236444',
  '156625',
  '283505',
  '283392',
  '39894',
  '215861',
  '276946',
  '123818'],
 0.08575003406711841: ['337143',
  '156625',
  '285343',
  '236444',
  '39894',
  '160940',
  '236951',
  '202557',
  '215861'],
 0.4911332581792054: ['214800', '156625', '160474', '160940', '39894'],
 0.06765420016599977: ['337143',
  '123818',
  '156625',
  '283392',
  '39894',
  '160940',
  '215861',
  '284096',
  '160474'],
 0.6384766423448087: ['283

In [ ]:
recall
15597.164410589485
precision
0.10132063294911522
NDGC
0.4718336765044949
CTR
0.43422137013314516
59935
93073

In [3]:
from math import log

In [4]:
import pandas as pd
df = pd.read_csv('news_dataset_last_40_encoded.csv')

In [5]:
df.head()

,user_id,session_id,session_start,session_size,click_timestamp,click_region,created_at_ts,words_count,article_id,category_26,...,click_country_9,click_country_10,click_country_11,click_referrer_type_1,click_referrer_type_2,click_referrer_type_3,click_referrer_type_4,click_referrer_type_5,click_referrer_type_6,click_referrer_type_7
0,278760,1507738846274401,1507738846000,2,1507740326434,5,1507633764000,175,29953,1,...,0,0,0,1,0,0,0,0,0,0
1,2522,1507678682105944,1507678682000,14,1507688543249,21,1507633764000,175,29953,1,...,0,0,0,1,0,0,0,0,0,0
2,27768,1507678812218062,1507678812000,4,1507679608544,9,1507633764000,175,29953,1,...,0,0,0,1,0,0,0,0,0,0
3,30545,1507678889494130,1507678889000,2,1507679382763,16,1507633764000,175,29953,1,...,0,0,0,1,0,0,0,0,0,0
4,114574,1507679057319269,1507679057000,5,1507679399082,25,1507633764000,175,29953,1,...,0,0,0,1,0,0,0,0,0,0


In [6]:
df=df.drop(columns =['session_id','session_start','created_at_ts'])

In [7]:
df = df.sample(frac=1)

In [8]:
df =df.drop(columns =['click_timestamp'])

In [9]:
def preprocess(df):
    from sklearn.preprocessing import minmax_scale

    df[['session_size','click_region','words_count']] = minmax_scale(df[['session_size','click_region','words_count']])
    return df

In [10]:
df = preprocess(df)

In [11]:
df.head()

,user_id,session_size,click_region,words_count,article_id,category_26,category_66,category_136,category_186,category_209,...,click_country_9,click_country_10,click_country_11,click_referrer_type_1,click_referrer_type_2,click_referrer_type_3,click_referrer_type_4,click_referrer_type_5,click_referrer_type_6,click_referrer_type_7
177006,97422,0.008197,0.259259,0.389744,337143,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
48899,157730,0.000000,0.259259,0.507692,124176,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
44732,37164,0.049180,0.444444,0.630769,123826,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
53593,35326,0.008197,0.444444,0.794872,140528,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
156032,14814,0.040984,0.888889,0.646154,285343,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [12]:
df.to_csv('news_user.csv',index=None)